In [1]:
import pickle
import networkx as nx

# === Parameters ===
GPKL_FILE = "../highway_network/uk_driving_graph_simplified.gpickle"
TIME_WEIGHT_KEY = "travel_time"

# ---------------------------
# travel_time estimator
# ---------------------------
def estimate_speed_kph(tags):
    maxspeed = tags.get("maxspeed")
    if isinstance(maxspeed, str):
        speeds = []
        for p in maxspeed.split(","):
            for token in p.split():
                try:
                    speeds.append(float(token))
                    break
                except ValueError:
                    continue
        if speeds:
            return (sum(speeds)/len(speeds)) * 1.60934  # mph->kph ×90%

    uk_default_speeds_mph = {
        "motorway": 70, "motorway_link": 60,
        "trunk": 60, "trunk_link": 50,
        "primary": 45, "primary_link": 40,
        "secondary": 35, "secondary_link": 30,
        "tertiary": 25, "tertiary_link": 20,
        "residential": 15, "unclassified": 15, "service": 15,
    }
    hval = tags.get("highway", "unclassified")
    if isinstance(hval, str):
        htypes = [s.strip() for s in hval.split(",") if s.strip()]
    else:
        htypes = ["unclassified"]
    speeds = [uk_default_speeds_mph[h] for h in htypes if h in uk_default_speeds_mph]
    avg_mph = (sum(speeds)/len(speeds)) if speeds else 30
    return avg_mph * 1.60934

def compute_travel_time(G):
    filled = 0
    for _, _, _, data in G.edges(keys=True, data=True):
        if TIME_WEIGHT_KEY in data:
            continue
        length = data.get("length")
        if length is None:
            continue
        speed_kph = estimate_speed_kph(data)
        speed_mps = speed_kph * 1000 / 3600
        if speed_mps <= 0:
            continue
        data[TIME_WEIGHT_KEY] = float(length) / float(speed_mps)
        filled += 1
    print(f"Filled {filled} edges with computed '{TIME_WEIGHT_KEY}'.")

# ---------------------------
# Main
# ---------------------------
with open(GPKL_FILE, "rb") as f:
    G: nx.MultiDiGraph = pickle.load(f)

print("🕒 Ensuring travel time on all edges…")
compute_travel_time(G)
print("✅ Travel time ready.")

out_file = GPKL_FILE.replace(".gpickle", "_travel_time_added.gpickle")
with open(out_file, "wb") as f:
    pickle.dump(G, f, protocol=pickle.HIGHEST_PROTOCOL)

print(f"💾 Saved updated graph to: {out_file}")

🕒 Ensuring travel time on all edges…
Filled 7220434 edges with computed 'travel_time'.
✅ Travel time ready.
💾 Saved updated graph to: ../highway_network/uk_driving_graph_simplified_travel_time_added.gpickle
